In [1]:
from cnmodel import populations
from cnmodel.util import sound, random_seed
from cnmodel.protocols import Protocol
from collections import OrderedDict
import os, sys, time
import pickle
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from neuron import h
import multiprocessing as mp
import yaml

--No graphics will be displayed.
[2855536] Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2855536] Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2855536] NumExpr defaulting to 16 threads.


In [2]:
seed = 34657845
random_seed.set_seed(seed)

34657845

In [3]:
sgc = populations.SGC(model='dummy')
dstellate = populations.DStellate()
vertical = populations.Tuberculoventral()
pyramidal = populations.Pyramidal()

In [5]:
sgc.connect(pyramidal, dstellate, vertical)
dstellate.connect(pyramidal)
vertical.connect(pyramidal)

In [8]:
fmin = 4e3
fmax = 32e3
octavespacing = 1 / 8.0  # 8.0
n_frequencies = int(np.log2(fmax / fmin) / octavespacing) + 1
fvals = (
    np.logspace(
        np.log2(fmin / 1000.0),
        np.log2(fmax / 1000.0),
        num=n_frequencies,
        endpoint=True,
        base=2,
    )
    * 1000.0
)
fvals

array([ 4000.        ,  4362.03093066,  4756.82846001,  5187.3582186 ,
        5656.85424949,  6168.84330163,  6727.17132203,  7336.03234564,
        8000.        ,  8724.06186132,  9513.65692002, 10374.71643721,
       11313.70849898, 12337.68660326, 13454.34264406, 14672.06469127,
       16000.        , 17448.12372264, 19027.31384004, 20749.43287442,
       22627.41699797, 24675.37320653, 26908.68528812, 29344.12938255,
       32000.        ])

In [10]:
pyramidal_cell_ids = []
for fval in fvals:
    pyramidal_cell_ids.extend(pyramidal.select(1, cf=fval, create=True))
len(pyramidal_cell_ids)


25

In [11]:
pyramidal.resolve_inputs(depth=2)

getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpars: modelType:  TVmouse
getcellpar

In [13]:
for pop in [sgc, dstellate, vertical, pyramidal]:
    print(f'{pop.type} - {len(pop.real_cells())} cells')

sgc - 3319 cells
dstellate - 221 cells
tuberculoventral - 470 cells
pyramidal - 25 cells


In [14]:
level = 40
stimpar = {
    'level': 40,
    'freq': 10e3,
    "dur": 0.2,
    "pip": 0.04,
    "start": [0.1],
    "baseline": [50, 100],
    "response": [100, 140],
}

In [ ]:
stim = sound.TonePip(
    rate=100e3,
    duration=stimpar["dur"],
    f0=stimpar['freq'],
    dbspl=stimpar['level'],  # dura 0.2, pip_start 0.1 pipdur 0.04
    ramp_duration=2.5e-3,
    pip_duration=stimpar["pip"],
    pip_start=stimpar["start"]
)